In [1]:
library(ggplot2)
library(dplyr)
library(tidyr)
df <- read.csv("vase_daily.csv")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
cdf <- df %>%
    dplyr::arrange(Date) %>%
    dplyr::mutate(prop = (N_dead + N_dying) / N_total) %>%
    dplyr::group_by(Vase) %>%
    dplyr::mutate(obs_id = paste0("obs_", row_number())) %>%
    dplyr::ungroup() %>%
    dplyr::select(Vase, obs_id, prop) %>%
    tidyr::pivot_wider(names_from = obs_id, values_from = prop)

In [9]:
cdf_long <- cdf %>%
  pivot_longer(
    cols = starts_with("obs_"),
    names_to = "Observation",
    values_to = "Value"
  ) %>%
  mutate(Observation = rep(c(0, 2, 3, 4),4))

p <- ggplot(cdf_long, aes(x = Observation, y = 100 * Value, color = Vase, group = Vase)) +
  geom_line(linewidth = 1) + 
  geom_point(size = 2) +
  scale_x_continuous(breaks = c(1, 2, 3, 4)) +
  labs(
    title = "Proportion of dying/dead flowers",
    x = "Time (days)",
    y = "Proportion (%)"
  ) +
  theme_minimal() + 
  scale_color_discrete(
    name = "Preservative-Water", 
    labels = c("Yes-Filtered", "No-Filtered", "Yes-Tap", "No-Tap") 
  )

ggsave(
  filename = "survival.pdf", 
  plot = p,                     
  device = "pdf",                     
  width = 4,                      
  height = 3,                        
  units = "in"                      
)